In [0]:
# Install gspread, authenticate and load data from a Google Sheet
!pip install --upgrade -q gspread

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

# Default data from http://web.eecs.utk.edu/~berry/order/node4.html#SECTION00022000000000000000

googlesheet_filename = '13_topic_modelling_data'
data_rows_to_preview = 10


In [3]:
#Load and preview data from a Google Sheet

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open(googlesheet_filename).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# convert the 2nd column values to a list
documents = []
for row in rows[1:]:
  documents.append(row[1])
  
#print(documents)

# Convert to a DataFrame and render.
import pandas as pd
dataset_df = pd.DataFrame.from_records(rows)
dataset_df.head(n=data_rows_to_preview)


,0,1
0,id,text
1,1,"Avram Noam Chomsky (Filadelfia, 7 de diciembre..."
2,2,"Propuso la gramática generativa, disciplina qu..."
3,3,Destaca su contribución al establecimiento de ...


In [0]:
#Set topic modeling algorithm arguments

no_topics = 3

no_top_words = 4

no_top_documents = 3

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np

In [5]:
!pip install nltk

In [0]:
import nltk

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
from nltk.corpus import stopwords

In [9]:
# stopwords = [word.decode('utf-8') for word in stopwords.words('spanish')]
# stopwords = [word for word in stopwords.words('spanish')]
print(stopwords.words('spanish')[:20])

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo']


In [10]:
# Run NMF

def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
  for topic_idx, topic in enumerate(H):
    print("Topic %d:" % (topic_idx))
    print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
    for doc_index in top_doc_indices:
      print(documents[doc_index])

# from sklearn.feature_extraction import text
# stop_words = text.ENGLISH_STOP_WORDS.union(stopwords)
  
# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stopwords.words("spanish"))
# tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words="english")
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# Run NMF
nmf_model = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)
nmf_W = nmf_model.transform(tfidf)
nmf_H = nmf_model.components_

print("NMF Topics")
display_topics(nmf_H, nmf_W, tfidf_feature_names, documents, no_top_words, no_top_documents)
print("--------------")



NMF Topics
Topic 0:
lenguaje estudio ciencias lingüística
Propuso la gramática generativa, disciplina que situó la sintaxis en el centro de la investigación lingüística. Con ésta cambió la perspectiva, los programas y métodos de investigación en el estudio del lenguaje. Su lingüística es una teoría de la adquisición individual del lenguaje e intenta explicar las estructuras y principios más profundos del lenguaje. Postuló un aspecto bien definido de innatismo en la adquisición del lenguaje y la autonomía de la gramática (sobre los otros sistemas cognitivos), así como la existencia de un «órgano del lenguaje» y de una gramática universal. Se opuso con dureza al empirismo filosófico y científico y al funcionalismo, en favor del racionalismo cartesiano. Todas estas ideas chocaban frontalmente con las tradicionales de las ciencias humanas, lo que concitó múltiples adhesiones, críticas y polémicas que le han acabado convirtiendo en uno de los autores más citados.
Destaca su contribución al 

In [14]:
# Run LDA

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words=stopwords.words("spanish"))
# tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

# Run LDA
lda_model = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

print("LDA Topics")
display_topics(lda_H, lda_W, tf_feature_names, documents, no_top_words, no_top_documents)

LDA Topics
Topic 0:
crítica chomsky lingüística ciencias
Avram Noam Chomsky (Filadelfia, 7 de diciembre de 1928) es un lingüista, filósofo, politólogo y activista estadounidense. Es profesor emérito de lingüística en el Instituto Tecnológico de Massachusetts (MIT) y una de las figuras más destacadas de la lingüística del siglo XX, gracias a sus trabajos en teoría lingüística y ciencia cognitiva. También es reconocido por su activismo político, caracterizado por una fuerte crítica del capitalismo contemporáneo y de la política exterior de los Estados Unidos. Se le considera de pensamiento socialista libertario.2​ El New York Times lo ha señalado como «el más importante de los pensadores contemporáneos»
Propuso la gramática generativa, disciplina que situó la sintaxis en el centro de la investigación lingüística. Con ésta cambió la perspectiva, los programas y métodos de investigación en el estudio del lenguaje. Su lingüística es una teoría de la adquisición individual del lenguaje e int